In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('new_dataset.csv')
data = data.apply(lambda x: x.str.replace(r'0 \(Yeni\)', '0', regex=True) if x.dtype == "object" else x)
data = data.apply(lambda x: x.str.replace(r'Ve Üzeri', '', regex=True) if x.dtype == "object" else x)
sütunlar = [
    'Yapı Durumu', 'Krediye Uygunluk', 'Isıtma Tipi', 
    'Site İçerisinde', 'Kullanım Durumu', 'Yatırıma Uygunluk', 
    'Banyo Sayısı', 'WC Sayısı', 'Eşya Durumu', 'Tapu Durumu', 
    'Balkon Sayısı', 'Tipi', 'Unnamed: 0'
]       
data.drop(columns=sütunlar, inplace=True, errors='ignore')

mahalle_sutun_adi = 'Mahalle'
unique_mahalleler = data[mahalle_sutun_adi].unique()
mahalle_dict = {mahalle: idx for idx, mahalle in enumerate(unique_mahalleler)}
data['Mahalle'] = data[mahalle_sutun_adi].map(mahalle_dict)

ilce_sutun_adi = 'Ilce'
unique_ilceler = data[ilce_sutun_adi].unique()
ilce_dict = {ilce: idx for idx, ilce in enumerate(unique_ilceler)}
data['Ilce'] = data[ilce_sutun_adi].map(ilce_dict)

sehir_sutun_adi = 'Sehir'
unique_sehirler = data[sehir_sutun_adi].unique()
sehir_dict = {sehir: idx for idx, sehir in enumerate(unique_sehirler)}
data['Sehir'] = data[sehir_sutun_adi].map(sehir_dict)

data

,Fiyat,Oda Sayısı,Bulunduğu Kat,Brüt Metrekare,Binanın Yaşı,Binanın Kat Sayısı,Sehir,Ilce,Mahalle
0,1240000,3,5,87,21,5,0,0,0
1,4800000,3,1,120,11-15,1,0,0,1
2,2250000,4,4,160,16-20,7,0,0,2
3,3000000,4,1,343,5-10,2,0,0,1
4,2500000,3,1,269,5-10,1,0,0,3
...,...,...,...,...,...,...,...,...,...
119878,1250000,3,4,105,5-10,4,31,405,1574
119879,3300000,4,3,130,0,3,31,405,3282
119880,1300000,5,2,140,21,3,31,405,3282
119881,1250000,3,2,90,5-10,3,31,405,160


In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()
data[['Brüt Metrekare', 'Oda Sayısı']] = scaler.fit_transform(data[['Brüt Metrekare', 'Oda Sayısı']])
data['Kat Oranı'] = data['Bulunduğu Kat'] / data['Binanın Kat Sayısı']
data = data.drop('Bulunduğu Kat', axis=1)
data = data.drop('Binanın Kat Sayısı', axis=1)
def convert_age(age):
    if '-' in str(age):
        age_range = age.split('-')
        return (int(age_range[0]) + int(age_range[1])) / 2
    else:
        return float(age)

data['Binanın Yaşı'] = data['Binanın Yaşı'].apply(convert_age)

data

,Fiyat,Oda Sayısı,Brüt Metrekare,Binanın Yaşı,Sehir,Ilce,Mahalle,Kat Oranı
0,1240000,-0.619940,-0.827398,21.0,0,0,0,1.000000
1,4800000,-0.619940,-0.207387,13.0,0,0,1,1.000000
2,2250000,0.412185,0.544141,18.0,0,0,2,0.571429
3,3000000,0.412185,3.982384,7.5,0,0,1,0.500000
4,2500000,-0.619940,2.592056,7.5,0,0,3,1.000000
...,...,...,...,...,...,...,...,...
119878,1250000,-0.619940,-0.489210,7.5,31,405,1574,1.000000
119879,3300000,0.412185,-0.019505,0.0,31,405,3282,1.000000
119880,1300000,1.444310,0.168377,21.0,31,405,3282,0.666667
119881,1250000,-0.619940,-0.771033,7.5,31,405,160,0.666667


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

columns_to_scale = ['Oda Sayısı', 'Brüt Metrekare', 'Binanın Yaşı', 'Sehir', 'Ilce', 'Mahalle', 'Kat Oranı']
scaler = StandardScaler()
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

X = data[['Oda Sayısı', 'Brüt Metrekare', 'Binanın Yaşı', 'Sehir', 'Ilce', 'Mahalle', 'Kat Oranı']]
y = data['Fiyat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test[columns_to_scale] = scaler.fit_transform(X_test[columns_to_scale])

model = SVR(kernel='rbf', C=1.0, gamma=0.1)
model.fit(X_train, y_train)